In [1]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

from os.path import join

In [2]:
ROOT_DIR = 'f:/datasets/NATUR_CUNI'
year=2021
inpath_vector = join(ROOT_DIR, f'reference/{year}/vyjezdy_{year}_zonal_stats.gpkg')

In [3]:
class Time_series:
    def __init__(self, inpath_vector):
        self.sar_gdf = gpd.read_file(inpath_vector)
        self.sar_keys = [key for key in self.sar_gdf.keys() if 'VV_' in key or 'VH_' in key]
        self.sar_dates = list(set([datetime.strptime(key[3:12], '%Y_%m%d') for key in self.sar_keys]))
        self.sar_dates.sort()
        self.metrics = self._extract_metrics()
        
        self.obs_gdf = self.sar_gdf.filter(regex='\A\d.*\d\Z', axis='columns')
        self.obs_dates = [datetime.strptime(f'2021_{date}', '%Y_%d_%m') for date in self.obs_gdf.keys()]
        
        
    def _extract_metrics(self):
        pols = set([i[:2] for i in self.sar_keys])
        stats = set([i[18:] for i in self.sar_keys])
        
        metrics = {}
        for stat in stats:
            for pol in pols:
                cols = [coh for coh in self.sar_keys if stat in coh and pol in coh]
                cols.sort()
                metrics[f'{stat}_{pol}'] = cols
        return metrics
        
    def list_metrics(self):
        print(self.metrics.keys())
    
    def plot_series_single(self, plot_idx=0, metric='mean_VH'):
        fig, ax1 = plt.subplots()

        ax1.set_xlabel('Time')
        ax1.set_ylabel('S1 Coherence', color = 'red')
        ax1.set_ylim(0, 1)
        
        y = self.sar_gdf.iloc[plot_idx].loc[self.metrics[metric]]
        
        plot_1 = ax1.plot(self.sar_dates, y, color='red')
        std = f'st_{metric[-2:]}'
        if std in self.metrics:
            y_bottom = y - self.sar_gdf.iloc[plot_idx].loc[self.metrics[std]].to_numpy()
            y_top =    y + self.sar_gdf.iloc[plot_idx].loc[self.metrics[std]].to_numpy()
            dict_df = {
                'x': self.sar_dates,
                'y_bottom': y_bottom,
                'y_top': y_top,
            }
            df = pd.DataFrame(dict_df)
            pd.to_datetime(df['x'],format='%Y%m%d')
           
            print(self.sar_gdf.iloc[plot_idx].loc[self.metrics[std]])
            print(y)
            print(y_bottom)
            print(y_top)
            print(df)
            plot_1 = ax1.fill_between(data=df, x='x', y1='y_bottom', y2='y_top')
        
        plot_1 = ax1.plot(self.sar_dates, y, color='red')
        ax1.tick_params(axis ='y', labelcolor = 'red')

        # Adding Twin Axes
        ax2 = ax1.twinx()
        ax2.set_ylabel('In situ grass height [cm]', color = 'green')
        plot_2 = ax2.bar(self.obs_dates, self.obs_gdf.iloc[plot_idx], color='green')
        ax2.tick_params(axis ='y', labelcolor = 'green')

        # Show plot
        plt.show()

    def plot_series_all(self, metric='mean_VH'):
        fig, (axs) = plt.subplots(ncols=6, nrows=10, squeeze=False, sharex='all', sharey='all')
        axs_flat = axs.flatten()

        for idx, row in enumerate(self.sar_gdf.iterrows()):
            x = [datetime.strptime(string[3:12], '%Y_%m%d') for string in row[1][self.metrics[metric]].keys()]
            y = row[1][self.metrics[metric]]
            axs_flat[idx].plot(x, y)

In [4]:
hello_world = Time_series(inpath_vector)

In [5]:
hello_world.list_metrics()

dict_keys(['std_VH', 'std_VV', 'count_VH', 'count_VV', 'median_VH', 'median_VV', 'mean_VH', 'mean_VV'])


In [ ]:
hello_world.plot_series_single(15, 'median_VH')

In [ ]:
hello_world.plot_series_all()

In [ ]:
'mean_VH'[-2:]